# Magic Facies Prediction

In [1]:
import pandas as pd
from matplotlib import pyplot as plt
%matplotlib inline
from sklearn.preprocessing import PolynomialFeatures
from sklearn import linear_model
from sklearn.linear_model import LogisticRegression
from sklearn.metrics import accuracy_score
import sys
# Import our custom modules
sys.path.append("..")
from mysticbit import ml, munging, plots

Do you really want to drill me?
Do you really want to make me cry
AH AH AH AH


### Data Loading & Preparation

In [2]:
df = munging.load_log_data()
df.head()

,PSEUDO_DEPTH,TEMP,CALI,GR,ILD,NPHI,RHOB,DT,TVD,TVDSS,...,PHIE,PHIT,SW,EF,PERFOS,RES_ID,WELL_ID,X,Y,HACKANAME
0,2074.53,87.19,10.31,55.08,3.3128,0.1678,2.3124,108.95,1588.62,1566.62,...,0.0591,0.2066,0.5863,5,0,3005,210075859,15,20.0,B03
1,2074.68,87.20,10.25,52.89,3.6592,0.1739,2.2895,110.91,1588.77,1566.77,...,0.0871,0.2222,0.5225,5,0,3005,210075859,15,20.0,B03
2,2074.83,87.20,10.20,52.54,4.1220,0.1618,2.2686,114.92,1588.92,1566.92,...,0.0730,0.2061,0.5004,5,0,3005,210075859,15,20.0,B03
3,2074.98,87.21,10.18,54.36,4.7189,0.1448,2.2533,119.66,1589.06,1567.06,...,0.0335,0.1768,0.4566,5,0,3005,210075859,15,20.0,B03
4,2075.90,87.24,9.76,48.13,13.0483,0.1085,2.0885,126.26,1589.94,1567.94,...,0.0331,0.1412,0.2211,1,1,3005,210075859,15,20.0,B03


In [3]:
print(f"Titles: {' '.join(list(df))}")

Titles: PSEUDO_DEPTH TEMP CALI GR ILD NPHI RHOB DT TVD TVDSS VCL PHIE PHIT SW EF PERFOS RES_ID WELL_ID X Y HACKANAME


In [4]:
facies = ml.create_facies(df, 4)